In [1]:
import numpy as np
categories = np.load('./categories.npy')
valid_periods = np.load('./valid_periods.npy')
training_data = np.load('./training_data.npy')

In [2]:
import pandas as pd
training_data_df = pd.DataFrame(training_data)
training_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Columns: 2776 entries, 0 to 2775
dtypes: float64(2776)
memory usage: 1016.6 MB


In [3]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
reshaped_categories = categories.reshape(-1, 1)
onehot_categories = encoder.fit_transform(reshaped_categories)

/home/ubuntu/hive_retrieval_engine/test_venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
from sklearn.model_selection import train_test_split

# Split the indices of the time series into training and validation sets
train_indices, val_indices = train_test_split(np.arange(len(training_data_df)), test_size=0.2, random_state=42, stratify=categories)

# Use these indices to create training and validation sets
train_set = training_data_df.iloc[train_indices]
valid_periods_train = valid_periods[train_indices]
onehot_categories_train = onehot_categories[train_indices]

val_set = training_data_df.iloc[val_indices]
valid_periods_val = valid_periods[val_indices]
onehot_categories_val = onehot_categories[val_indices]

train_set.shape, valid_periods_train.shape, onehot_categories_train.shape, val_set.shape, valid_periods_val.shape, onehot_categories_val.shape

((46560, 2776), (46560, 2), (46560, 6), (1440, 2776), (1440, 2), (1440, 6))

In [13]:
def calculate_padding_length(sq, w, t, s):
    cycle = w + t
    if sq <= cycle: return cycle - sq
    else:
        m = (sq - cycle) % s
        if m == 0: return m
        else: return s - m

In [14]:
def create_sequences(df, valid_periods, categories, window, telescope, stride=1):

    input_sequences = []
    input_categories = []
    output_sequences = []


    for i in range(len(valid_periods)):
        start, end = valid_periods[i]
        category = categories[i]

        sequence = df.iloc[i, start:end].values

        padding_length = calculate_padding_length(len(sequence), window, telescope, stride)
        sequence = np.pad(sequence, (padding_length, 0), mode='constant', constant_values=0)

        for j in range(0, len(sequence) - window - telescope + 1, stride):
            input_seq = sequence[j:(j + window)]
            output_seq = sequence[(j + window):(j + window + telescope)]

            input_sequences.append(input_seq)
            input_categories.append(category)
            output_sequences.append(output_seq)

    return np.array(input_sequences), np.array(input_categories), np.array(output_sequences)

In [15]:
window = 100
telescope = 9
stride = 10

In [16]:
X_train, X_train_category, y_train = create_sequences(train_set, valid_periods_train, onehot_categories_train, window, telescope, stride)
X_val, X_val_category, y_val = create_sequences(val_set, valid_periods_val, onehot_categories_val, window, telescope, stride)
X_train.shape, X_train_category.shape, y_train.shape, X_val.shape, X_val_category.shape, y_val.shape

((549460, 100), (549460, 6), (549460, 9), (17774, 100), (17774, 6), (17774, 9))

In [17]:
input_shape = (window, 1)
category_shape = (6, )
output_shape = (telescope, 1)
batch_size = 32
epochs = 200

In [18]:
import tensorflow.keras.layers as tfkl
import tensorflow as tf

def build_CONV_LSTM_model_with_categories(input_shape, category_shape, output_shape):

    # Time series input
    time_series_input = tfkl.Input(shape=input_shape, name='time_series_input')
    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, name='bidirectional_lstm'))(time_series_input)
    x = tfkl.Bidirectional(tfkl.LSTM(32, return_sequences=True, name='bidirectional_lstm_2'))(x)

    # Attention mechanism
    attention = tfkl.Attention(name='attention')([x, x])

    x = tfkl.Conv1D(128, 3, padding='same', activation='relu')(attention)
    x = tfkl.Dropout(0.2)(x)

    # Category input
    category_input = tfkl.Input(shape=category_shape, name='category_input')
    category_processed = tfkl.Dense(20, activation='relu')(category_input)

    # Combine the processed inputs
    combined = tfkl.Concatenate()([x, tfkl.RepeatVector(input_shape[0])(category_processed)])

    # Continue with convolutional layers
    x = tfkl.Conv1D(128, 3, padding='same', activation='relu')(combined)
    x = tfkl.Dropout(0.2)(x)

    output_layer = tfkl.Conv1D(output_shape[1], 3, padding='same')(x)
    crop_size = output_layer.shape[1] - output_shape[0]
    output_layer = tfkl.Cropping1D((0, crop_size))(output_layer)

    # Construct the model
    model = tf.keras.Model(inputs=[time_series_input, category_input], outputs=output_layer, name='CONV_LSTM_with_Categories')
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(1e-3))

    return model

In [19]:
model = build_CONV_LSTM_model_with_categories(input_shape, category_shape, output_shape)

In [20]:
model.summary()

from tensorflow import keras as tfk
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

Model: "CONV_LSTM_with_Categories"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 time_series_input (InputLa  [(None, 100, 1)]             0         []                            
 yer)                                                                                             
                                                                                                  
 bidirectional_2 (Bidirecti  (None, 100, 128)             33792     ['time_series_input[0][0]']   
 onal)                                                                                            
                                                                                                  
 bidirectional_3 (Bidirecti  (None, 100, 64)              41216     ['bidirectional_2[0][0]']     
 onal)                                                                    

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=15,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    patience=10,
    factor=0.1,
    min_lr=1e-5
)

history = model.fit(
    x = [X_train, X_train_category],
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = ([X_val, X_val_category], y_val),
    callbacks = [
        early_stopping,
        reduce_lr
    ]
).history

Epoch 1/200


2023-12-19 14:59:48.832164: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla T4" frequency: 1590 num_cores: 40 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11080" } environment { key: "cudnn" value: "8600" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 14298316800 bandwidth: 320064000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-12-19 14:59:51.995629: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-12-19 14:59:58.820519: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555fbcd2e060 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-19 1

14154/14154 [==============================] - ETA: 0s - loss: 0.0137

2023-12-19 15:03:26.386569: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla T4" frequency: 1590 num_cores: 40 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11080" } environment { key: "cudnn" value: "8600" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 14298316800 bandwidth: 320064000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


14154/14154 [==============================] - 244s 16ms/step - loss: 0.0137 - val_loss: 0.0095 - lr: 0.0010
Epoch 2/200
14154/14154 [==============================] - 220s 16ms/step - loss: 0.0100 - val_loss: 0.0090 - lr: 0.0010
Epoch 3/200
14154/14154 [==============================] - 220s 16ms/step - loss: 0.0090 - val_loss: 0.0081 - lr: 0.0010
Epoch 4/200
13871/14154 [============================>.] - ETA: 3s - loss: 0.0085

In [30]:
for i, item in enumerate(history['loss']):
    print(f"Epoch {i}: {history['loss'][i]:.5f}, {history['val_loss'][i]:.5f}")

Epoch 0: 0.01369, 0.00952
Epoch 1: 0.00998, 0.00899
Epoch 2: 0.00903, 0.00806
Epoch 3: 0.00855, 0.00784
Epoch 4: 0.00897, 0.00958
Epoch 5: 0.00909, 0.00786
Epoch 6: 0.00848, 0.00790
Epoch 7: 0.00819, 0.00768
Epoch 8: 0.00801, 0.00848
Epoch 9: 0.00794, 0.00752
Epoch 10: 0.00863, 0.00886
Epoch 11: 0.00831, 0.00783
Epoch 12: 0.00785, 0.00735
Epoch 13: 0.00771, 0.00747
Epoch 14: 0.00764, 0.00756
Epoch 15: 0.00757, 0.00728
Epoch 16: 0.00750, 0.00744
Epoch 17: 0.00746, 0.00726
Epoch 18: 0.00741, 0.00718
Epoch 19: 0.00736, 0.00711
Epoch 20: 0.00732, 0.00757
Epoch 21: 0.00727, 0.00699
Epoch 22: 0.00723, 0.00715
Epoch 23: 0.00720, 0.00711
Epoch 24: 0.00717, 0.00714
Epoch 25: 0.00714, 0.00701
Epoch 26: 0.00711, 0.00696
Epoch 27: 0.00707, 0.00703
Epoch 28: 0.00705, 0.00714
Epoch 29: 0.00702, 0.00698
Epoch 30: 0.00700, 0.00694
Epoch 31: 0.00697, 0.00721
Epoch 32: 0.00660, 0.00685
Epoch 33: 0.00654, 0.00692
Epoch 34: 0.00651, 0.00698
Epoch 35: 0.00650, 0.00692
Epoch 36: 0.00648, 0.00709
Epoch 37: 0

In [ ]:
import matplotlib as plt
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [16]:
model.save('conv_lstm_tp9_ws100_att')

INFO:tensorflow:Assets written to: conv_lstm_tp9_ws100_att/assets


INFO:tensorflow:Assets written to: conv_lstm_tp9_ws100_att/assets


In [ ]:
!zip model.zip -r conv_lstm_tp9_ws100_att/